<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Images Hope

Goal : have well trained models for image sampling

Data : celeba64,mnist,svhn,celeba32

Models : ConvLNPXL 

Loss : NLLLloss

Runs : 1

In [11]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

In [3]:

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [17]:
args = get_exp_args("exp_hope", is_load=False)
len(args)

1

In [6]:
executor_32=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor_32.update_parameters(num_gpus=1, 
                           time=60*24*1,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips"
                          )

In [7]:
jobs= executor_32.map_array(Run(), args)

In [8]:
############################################################

In [26]:
jobs

[SlurmJob<job_id=26817460_0, task_id=0, state="COMPLETED">]

In [25]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-26 00:34:50,944) - Starting with JobEnvironment(job_id=26817460_0, hostname=learnfair0493, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-26 00:34:50,944) - Loading pickle: /private/home/yannd/projects/NPF/logs/26817460_0/26817460_0_submitted.pkl

--- Training mnist/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1     -927.6429    -1411.4283     +  955.6045
      2    -1353.7954    -1472.4902     +  954.2092
      3    -1562.5733    -1433.8629        955.0092
      4    -1679.3929    -1788.0330     +  954.4729
      5    -1784.0745    -1865.1760     +  955.1132
      6    -1854.9314    -1899.0963     +  955.0936
      7    -1902.9332    -1938.5072     +  955.3696
      8    -1950.9389    -1945.1159     +  954.7381
      9    -1975.5394    -2057.5726     +  955.2038
     10    -2005.1842    -1996.8774        947.3034
  

In [54]:
for j in jobs_32:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-05-24 20:41:35,817) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submi

In [21]:
for job in jobs_64:
    job.cancel()